# Graph Reordering: Julia

It is almost always worth tryin to reorder matrices. 

When you first use a non-library package you need to load it 
and tell Julia where to find it. It should just load

In [44]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/rleegates/CuthillMcKee.jl.git"))

     Cloning git-repo `https://github.com/rleegates/CuthillMcKee.jl.git`
    Updating git-repo `https://github.com/rleegates/CuthillMcKee.jl.git`
   Resolving package versions...
    Updating `C:\Users\struther\.julia\environments\v1.6\Project.toml`
  [17f17636] + CuthillMcKee v0.1.0 `https://github.com/rleegates/CuthillMcKee.jl.git#master`
    Updating `C:\Users\struther\.julia\environments\v1.6\Manifest.toml`
  [17f17636] + CuthillMcKee v0.1.0 `https://github.com/rleegates/CuthillMcKee.jl.git#master`
Precompiling project...
  ✓ CuthillMcKee
  1 dependency successfully precompiled in 4 seconds (179 already precompiled)


In [48]:
using SparseArrays, CuthillMcKee, UnicodePlots
N = 5000
A = sprand(N, N, 1/N)
A = A+A'
@time p = symrcm(A)
AP = rcmpermute(A)
display(spy(A))
display(spy(AP))

        ┌──────────────────────────────────────────┐    
      1 │⡿⣯⣹⢏⠷⣻⠹⣯⣿⣉⡻⢫⢿⣼⣟⡞⡞⣿⣿⣖⣿⣟⠟⣏⣓⣯⡯⣲⡿⣓⢖⣾⠟⡾⣽⢻⢏⣩⠷⣳⢯⣛│ > 0
        │⡷⢞⢺⣶⠟⣰⣾⣿⠾⢾⢟⣖⣧⣿⣿⡋⣿⠻⢧⣚⣟⣿⡞⠟⣦⣷⣻⢽⣵⡻⣲⡷⢦⣛⡾⣓⡮⣿⡾⣥⣛⡻│ < 0
        │⣽⣣⢛⣡⢪⣶⢿⣿⣳⡝⣯⢾⡣⣿⠳⣽⢿⢗⣯⢟⢘⡶⢵⢾⣟⣿⡛⢷⢹⣽⣞⡷⣛⣿⣮⣾⡿⢿⣞⣾⡮⠿│    
        │⡷⣦⣾⣿⣿⣷⣵⡻⢟⣿⣷⣿⢋⣹⢯⣞⣛⢮⣇⢸⣿⡺⣯⢞⡻⠿⣲⡿⢮⢻⡻⢕⡿⢗⣖⡿⣝⣷⣾⣳⡥⡾│    
        │⡟⢻⣺⣇⣝⠾⣿⣵⣯⡻⣼⣾⣩⣿⡿⡿⣾⢲⠿⡻⣙⣻⣕⠽⣵⣻⠾⡨⠵⡽⣧⠫⣵⣿⣟⢿⣿⣻⣟⣶⣮⣏│    
        │⡿⣊⢻⢵⣫⣟⣽⣿⣲⣿⣿⢛⣼⢩⣿⢗⡖⣏⡭⢮⣺⣆⣿⢻⢿⣿⡽⣿⠿⣝⡞⡽⡷⣾⣗⣢⣭⣻⣍⣉⣗⣿│    
        │⣛⣷⣭⣿⣭⣮⣏⣰⣧⣾⡖⣛⣎⡹⡻⡟⣿⡷⡿⣯⡿⣮⢷⡻⢍⣺⣾⡾⣟⣻⡟⡻⣽⡭⣿⣻⣿⡱⡶⠛⣿⣟│    
        │⣻⠽⡿⠻⣝⣦⣫⢷⣿⡯⢿⢟⣿⠮⣾⢛⢿⡻⣳⣾⣪⡝⣿⢾⣯⢪⣷⡫⣧⡫⡤⣽⡎⣽⡿⢟⣟⣿⣳⡢⣿⣟│    
        │⣾⣭⣿⡛⢿⢗⡻⣜⢺⣛⡼⢭⢿⡿⣿⡳⣴⣿⡝⣛⡟⣯⢴⣫⣽⢤⣿⣟⢿⣮⢺⠨⣵⡿⢟⣭⣾⣶⡾⢳⣶⣽│    
        │⢻⢿⣩⢳⣯⢟⣉⣙⣿⡣⡣⣏⡿⣯⣹⣾⣷⢩⣮⠛⣹⣷⡿⣝⣫⣮⡏⣇⣛⡣⡺⣿⢿⣞⣼⢕⡘⡮⣯⣞⢻⣿│    
        │⣿⢿⣿⣽⢲⡴⣻⡻⣷⣸⠺⢾⡻⣯⣎⠾⡿⣭⢷⣾⠤⡧⣗⣿⣏⣮⢍⣿⡷⠯⣯⡋⢷⢾⠞⣮⣶⣾⣟⣵⣜⢷│    
        │⡿⢥⣾⠍⣱⣗⣫⢟⣕⡝⣿⣛⣽⡳⣻⣟⡴⣳⣟⢯⣽⣽⣺⡾⢼⣷⣷⣝⡟⣍⣿⣿⢯⣸⢯⣟⢿⡻⢚⣏⣯⢯│    
        │⡽⣼⢬⣿⣿⣽⣿⡎⣵⣻⣿⣷⣣⣱⡫⣛⠓⣟⡫⣾⡫⣽⢶⣷⣯⡿⣟⡓⠶⣿⡷⣿⢜⣗⣫⢅⡯⢯⣩⡥⡯⢿│    
        │⢫⣫⣟⣞⢿⣌⣼⡾⡚⡣⣷⣯⣺⡿⡽⡻⣿⢿⠯⢭⣧⣵⣝⢿⢿⠹⣯⠟⣭⠧⣿⣞⣟⠾⣻⣳⡓⡾⣵⢮⣽⢫│    
        │⢿⢫⣵⡻⣗⣶⣮⣓⣕⡧⣟⢧⣿⣹⡭⡻⡻⣷⠿⡸⡽⡏⡟⢭⣼⣧⠧⡟⣮⡿⡾⣾⡗⣾⣻⣶⣟⣻⣿⣏⢋⣼│    
        │⣸⣵⢼⡾⢾⡽⢟⢎⡭⡛⣞⡭⣿⡩⣄⣯⡚⡒⣾⣮⡯⠻⣿⣿⣽⣯⣻⢿⣺⣯⡯⡯⣻⣹⢯⣔⣺⡳⣽⣿⣥⢽│    
        │⣻⡥⣬⢳⣿⣼⢿⢏⣵⣿⣹⣯⡗⡿⣎⣭⣵⡿⣻⢷⣹⣗

        ┌──────────────────────────────────────────┐    
      1 │⠱⢆⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
        │⠀⠈⠱⡦⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
        │⠀⠀⠀⠹⣯⣿⣿⡶⢤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⢻⡿⣿⡿⡷⣽⢷⢦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⢳⣝⣯⢴⡳⣻⣶⢙⢳⣦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠹⣗⢻⣾⠿⣧⢿⣟⣝⠿⣳⢄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠘⢷⣐⣿⢷⣮⡻⣯⣎⣿⡽⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠀⠈⢻⣷⡝⡫⢿⣎⣹⡟⣶⡪⢿⣝⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢞⣟⡿⢻⣭⣾⠛⣴⣟⣗⢿⢿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣮⣎⣴⢿⡛⣨⣧⣯⡷⢿⣿⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣝⣽⣝⡭⣿⣮⡿⢻⢿⡷⢾⢳⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣷⣽⣏⣿⣖⠫⡦⡯⣿⢿⣿⢦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢿⣹⣏⣯⣯⢺⡲⢼⠷⣛⣷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠲⣿⣷⢶⡗⢞⢑⢺⠿⡿⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠓⢿⣼⣾⡖⠻⣢⣐⣌⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢯⡐⢼⣴⡻⡟⣷⡀⠀⠀⠀⠀⠀⠀⠀│    
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

  0.000576 seconds (5.04 k allocations: 902.422 KiB)


# Questions

## How does it do this?

## How can I use this?

## When should I use this?